# 🎭 Facial Emotion Recognition - Model Training

This notebook provides a comprehensive pipeline for training a facial emotion recognition model using the FER2013 dataset.

## Contents:
1. **Setup & Dataset Download** - Install packages and download FER2013
2. **Data Loading & Exploration** - Explore dataset structure
3. **Data Visualization** - Class distribution and sample images
4. **Data Preprocessing** - Augmentation and generators
5. **Model Architecture** - Define CNN model
6. **Training** - Train the model with callbacks
7. **Evaluation** - Analyze model performance
8. **Export** - Save model in multiple formats

---
**Dataset**: FER2013 (48x48 grayscale images, 7 emotion classes)
- Emotions: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral
- Training samples: ~28,709
- Test samples: ~3,589

## 1. Setup and Dataset Download
Install required packages and download the FER2013 dataset from Kaggle using kagglehub.

In [ ]:
# Install required packages (run once)
# !pip install kagglehub tensorflow opencv-python matplotlib seaborn scikit-learn

In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout, Flatten,
    BatchNormalization, GlobalAveragePooling2D, Input
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, 
    ReduceLROnPlateau, TensorBoard
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Sklearn for evaluation
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Download dataset
import kagglehub

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Download FER2013 dataset using kagglehub
dataset_path = kagglehub.dataset_download("msambare/fer2013")
print(f"Dataset downloaded to: {dataset_path}")

# Define paths
TRAIN_DIR = os.path.join(dataset_path, 'train')
TEST_DIR = os.path.join(dataset_path, 'test')

# Create models directory for saving
MODELS_DIR = Path('../models')
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"\nTrain directory: {TRAIN_DIR}")
print(f"Test directory: {TEST_DIR}")
print(f"Models directory: {MODELS_DIR.absolute()}")

## 2. Data Loading and Exploration
Explore the dataset structure and count images per emotion category.

In [ ]:
# Define emotion classes
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
NUM_CLASSES = len(EMOTIONS)
IMG_SIZE = (48, 48)

# Count images per category
def count_images(directory):
    """Count images in each emotion subdirectory"""
    counts = {}
    for emotion in EMOTIONS:
        emotion_dir = os.path.join(directory, emotion)
        if os.path.exists(emotion_dir):
            counts[emotion] = len([f for f in os.listdir(emotion_dir) 
                                   if f.endswith(('.jpg', '.png', '.jpeg'))])
        else:
            counts[emotion] = 0
    return counts

# Get counts
train_counts = count_images(TRAIN_DIR)
test_counts = count_images(TEST_DIR)

# Create summary DataFrame
summary_df = pd.DataFrame({
    'Emotion': EMOTIONS,
    'Train': [train_counts[e] for e in EMOTIONS],
    'Test': [test_counts[e] for e in EMOTIONS]
})
summary_df['Total'] = summary_df['Train'] + summary_df['Test']
summary_df['Train %'] = (summary_df['Train'] / summary_df['Train'].sum() * 100).round(2)

print("=" * 60)
print("📊 DATASET SUMMARY")
print("=" * 60)
print(f"\n🏋️ Training samples: {summary_df['Train'].sum():,}")
print(f"🧪 Test samples: {summary_df['Test'].sum():,}")
print(f"📁 Total samples: {summary_df['Total'].sum():,}")
print(f"🎯 Number of classes: {NUM_CLASSES}")
print(f"📐 Image size: {IMG_SIZE}")
print("\n" + "=" * 60)
print(summary_df.to_string(index=False))
print("=" * 60)

## 3. Data Visualization - Class Distribution
Visualize the distribution of emotion classes in the dataset.

In [ ]:
# Set style for professional visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Color palette for emotions
emotion_colors = {
    'angry': '#FF6B6B',
    'disgust': '#4ECDC4',
    'fear': '#9B59B6',
    'happy': '#F39C12',
    'sad': '#3498DB',
    'surprise': '#E74C3C',
    'neutral': '#95A5A6'
}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Bar chart - Training distribution
colors = [emotion_colors[e] for e in EMOTIONS]
bars = axes[0].bar(EMOTIONS, [train_counts[e] for e in EMOTIONS], color=colors, edgecolor='white', linewidth=1.5)
axes[0].set_title('Training Set Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Emotion', fontsize=12)
axes[0].set_ylabel('Number of Images', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
for bar, count in zip(bars, [train_counts[e] for e in EMOTIONS]):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100, 
                 f'{count:,}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Pie chart - Training distribution
axes[1].pie([train_counts[e] for e in EMOTIONS], labels=EMOTIONS, autopct='%1.1f%%',
            colors=colors, explode=[0.02]*7, shadow=True, startangle=90)
axes[1].set_title('Training Set Proportion', fontsize=14, fontweight='bold')

# Grouped bar chart - Train vs Test
x = np.arange(len(EMOTIONS))
width = 0.35
bars1 = axes[2].bar(x - width/2, [train_counts[e] for e in EMOTIONS], width, label='Train', color='steelblue', alpha=0.8)
bars2 = axes[2].bar(x + width/2, [test_counts[e] for e in EMOTIONS], width, label='Test', color='coral', alpha=0.8)
axes[2].set_title('Train vs Test Distribution', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Emotion', fontsize=12)
axes[2].set_ylabel('Number of Images', fontsize=12)
axes[2].set_xticks(x)
axes[2].set_xticklabels(EMOTIONS, rotation=45)
axes[2].legend()

plt.tight_layout()
plt.savefig('../models/class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📈 Class distribution chart saved to models/class_distribution.png")

## 4. Data Visualization - Sample Images
Display sample images from each emotion category to understand the visual characteristics.

In [ ]:
import cv2
from PIL import Image

def load_sample_images(directory, emotion, n_samples=5):
    """Load sample images from an emotion directory"""
    emotion_dir = os.path.join(directory, emotion)
    images = []
    if os.path.exists(emotion_dir):
        files = [f for f in os.listdir(emotion_dir) if f.endswith(('.jpg', '.png', '.jpeg'))][:n_samples]
        for f in files:
            img_path = os.path.join(emotion_dir, f)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
    return images

# Create grid of sample images
fig, axes = plt.subplots(7, 6, figsize=(14, 16))
fig.suptitle('Sample Images per Emotion Class', fontsize=16, fontweight='bold', y=1.02)

for i, emotion in enumerate(EMOTIONS):
    samples = load_sample_images(TRAIN_DIR, emotion, n_samples=6)
    
    # Add emotion label on the left
    axes[i, 0].set_ylabel(emotion.upper(), fontsize=12, fontweight='bold', 
                          rotation=0, ha='right', va='center',
                          color=emotion_colors[emotion])
    
    for j, img in enumerate(samples):
        if j < 6:
            axes[i, j].imshow(img, cmap='gray')
            axes[i, j].axis('off')
    
    # Fill empty slots
    for j in range(len(samples), 6):
        axes[i, j].axis('off')

plt.tight_layout()
plt.savefig('../models/sample_images.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🖼️ Sample images grid saved to models/sample_images.png")

## 5. Data Preprocessing and Augmentation
Create data generators with augmentation for training and simple rescaling for validation.

In [ ]:
# Configuration
BATCH_SIZE = 64
TARGET_SIZE = IMG_SIZE  # (48, 48)

# Training data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation/Test data generator - only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

print("\n✅ Data generators created successfully!")
print(f"📦 Training batches: {len(train_generator)}")
print(f"📦 Validation batches: {len(validation_generator)}")
print(f"🏷️ Class indices: {train_generator.class_indices}")

In [ ]:
# Visualize augmented images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Data Augmentation Examples', fontsize=14, fontweight='bold')

# Get a batch of images
sample_batch = next(train_generator)
images, labels = sample_batch

for i in range(10):
    row, col = i // 5, i % 5
    axes[row, col].imshow(images[i].squeeze(), cmap='gray')
    emotion_idx = np.argmax(labels[i])
    axes[row, col].set_title(EMOTIONS[emotion_idx], fontsize=10)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

# Reset generator
train_generator.reset()

## 6. Model Architecture Definition
Define a CNN model with multiple convolutional blocks, batch normalization, and dropout for regularization.

In [ ]:
def build_emotion_model(input_shape=(48, 48, 1), num_classes=7):
    """
    Build a CNN model for facial emotion recognition.
    
    Architecture:
    - 4 Convolutional blocks with increasing filters
    - Batch normalization after each conv layer
    - MaxPooling and Dropout for regularization
    - Dense layers with dropout
    """
    model = Sequential([
        # Block 1
        Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Block 2
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Block 3
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Block 4
        Conv2D(512, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(512, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Fully Connected Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Build model
model = build_emotion_model(input_shape=(48, 48, 1), num_classes=NUM_CLASSES)

# Display model summary
model.summary()

# Visualize model architecture
print("\n📐 Model Architecture Summary:")
print(f"   Total layers: {len(model.layers)}")
print(f"   Total params: {model.count_params():,}")
print(f"   Trainable params: {sum([np.prod(w.shape) for w in model.trainable_weights]):,}")

## 7. Model Compilation and Callbacks
Compile the model and set up training callbacks for monitoring and optimization.

In [ ]:
# Compile the model
LEARNING_RATE = 0.0001

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Calculate class weights for imbalanced dataset
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weight_dict = dict(enumerate(class_weights))

print("📊 Class weights for imbalanced data:")
for idx, emotion in enumerate(EMOTIONS):
    print(f"   {emotion}: {class_weight_dict[idx]:.3f}")

In [ ]:
# Define callbacks
EPOCHS = 50

callbacks = [
    # Save best model
    ModelCheckpoint(
        filepath=str(MODELS_DIR / 'emotion_model_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Early stopping
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    # TensorBoard logging
    TensorBoard(
        log_dir=str(MODELS_DIR / 'logs'),
        histogram_freq=1
    )
]

print("✅ Callbacks configured:")
print("   - ModelCheckpoint: Save best model")
print("   - EarlyStopping: Stop if no improvement for 10 epochs")
print("   - ReduceLROnPlateau: Reduce LR by 0.5 if no improvement for 5 epochs")
print("   - TensorBoard: Training logs")

## 8. Model Training
Train the model with the configured callbacks and class weights.

In [ ]:
print("🚀 Starting training...")
print("=" * 60)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("=" * 60)
print("✅ Training completed!")

## 9. Training History Visualization
Plot training and validation accuracy/loss curves to analyze model learning.

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, color='steelblue')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, color='coral')
axes[0].set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(loc='lower right', fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2, color='steelblue')
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2, color='coral')
axes[1].set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(loc='upper right', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../models/training_history.png', dpi=150, bbox_inches='tight')
plt.show()

# Print best results
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
print(f"\n📈 Best validation accuracy: {best_val_acc:.4f} at epoch {best_epoch}")
print(f"📉 Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"📉 Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

## 10. Model Evaluation and Confusion Matrix
Evaluate the model on test data and generate detailed performance metrics.

In [ ]:
# Evaluate on test set
print("🧪 Evaluating model on test set...")
validation_generator.reset()
test_loss, test_accuracy = model.evaluate(validation_generator, verbose=1)

print(f"\n📊 Test Results:")
print(f"   Loss: {test_loss:.4f}")
print(f"   Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

In [ ]:
# Generate predictions
validation_generator.reset()
predictions = model.predict(validation_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_generator.classes

# Classification report
print("\n📋 Classification Report:")
print("=" * 60)
print(classification_report(true_classes, predicted_classes, target_names=EMOTIONS))

# Confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=EMOTIONS, yticklabels=EMOTIONS, ax=axes[0])
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('Actual', fontsize=12)

# Normalized
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=EMOTIONS, yticklabels=EMOTIONS, ax=axes[1])
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted', fontsize=12)
axes[1].set_ylabel('Actual', fontsize=12)

plt.tight_layout()
plt.savefig('../models/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Confusion matrix saved to models/confusion_matrix.png")

## 11. Model Export and Saving
Save the trained model in multiple formats for deployment.

In [ ]:
# Save model in different formats
print("💾 Saving model in multiple formats...")

# 1. Keras native format (recommended)
keras_path = MODELS_DIR / 'emotion_model.keras'
model.save(keras_path)
print(f"   ✅ Keras format: {keras_path}")

# 2. H5 format (legacy, for compatibility)
h5_path = MODELS_DIR / 'emotion_model.h5'
model.save(h5_path)
print(f"   ✅ H5 format: {h5_path}")

# 3. SavedModel format (for TensorFlow Serving)
savedmodel_path = MODELS_DIR / 'emotion_model_savedmodel'
model.save(savedmodel_path)
print(f"   ✅ SavedModel format: {savedmodel_path}")

# 4. Save model architecture to JSON
architecture_path = MODELS_DIR / 'model_architecture.json'
with open(architecture_path, 'w') as f:
    f.write(model.to_json())
print(f"   ✅ Architecture JSON: {architecture_path}")

# 5. Save training history
history_df = pd.DataFrame(history.history)
history_path = MODELS_DIR / 'training_history.csv'
history_df.to_csv(history_path, index=False)
print(f"   ✅ Training history: {history_path}")

# 6. Save model summary to text file
summary_path = MODELS_DIR / 'model_summary.txt'
with open(summary_path, 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    f.write(f"\n{'='*60}\n")
    f.write(f"Final Test Accuracy: {test_accuracy:.4f}\n")
    f.write(f"Final Test Loss: {test_loss:.4f}\n")
    f.write(f"Best Validation Accuracy: {best_val_acc:.4f} at epoch {best_epoch}\n")
print(f"   ✅ Model summary: {summary_path}")

print("\n🎉 All models and artifacts saved successfully!")
print(f"📁 Models directory: {MODELS_DIR.absolute()}")

In [ ]:
# Final summary
print("\n" + "=" * 60)
print("🏆 TRAINING COMPLETE - SUMMARY")
print("=" * 60)
print(f"""
📊 Dataset:
   - Training samples: {train_generator.samples:,}
   - Test samples: {validation_generator.samples:,}
   - Classes: {NUM_CLASSES} emotions

🧠 Model:
   - Architecture: Custom CNN (4 conv blocks)
   - Total parameters: {model.count_params():,}
   - Input shape: {IMG_SIZE} grayscale

📈 Performance:
   - Best validation accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)
   - Test accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)
   - Test loss: {test_loss:.4f}

💾 Saved Files:
   - emotion_model.keras (recommended)
   - emotion_model.h5 (legacy)
   - emotion_model_savedmodel/ (TF Serving)
   - model_architecture.json
   - training_history.csv
   - confusion_matrix.png
   - training_history.png

🚀 Next Steps:
   1. Run main.py to start real-time emotion detection
   2. The model will be loaded from models/emotion_model.keras
""")
print("=" * 60)